<center> <h1>Neural Network Model</h1> </center>

## 1. Reading data

We are reading data that we already prepared. We will look at the first few columns and we will choose 78. district to work on a small subset first.  
Finally we will use one hot encoding for categorical features then split the data for train and test.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
fileName = 'C:/Users/kazIm/Desktop/projects/IE490/input/tubitak_data2_processesed.csv'
df = pd.read_csv(fileName,  sep = ',')

In [ ]:
#preview data
df.head()

### Ilce 79

In [ ]:
df.drop(df.index[df.ilce_kod!=79],inplace=True)
# df
df.drop('ilce_kod',axis=1,inplace=True)

In [ ]:
df.info()

In [ ]:
mahalle = df["mahalle_kod"]

In [ ]:
df['mahalle_kod'].describe()

In [ ]:
#we can drop yasal burut alani as it has almost 1 correlation with mevcut alan
df = df.drop('yasal_burut_alani',axis=1)

In [ ]:
mahalle = df['mahalle_kod']

### One Hot Encoding for Categorical Variables

In [ ]:
# df = pd.get_dummies(df, columns=[""])
df = pd.get_dummies(df, columns=["mahalle_kod"])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# split into input (X) and output (y) variables
X = df.drop('adil_piyasa_degeri_yasal_durum',axis=1)
Y = df['adil_piyasa_degeri_yasal_durum']

In [ ]:
print(X.shape)
print(Y.shape)

## 2. Develop a Baseline Neural Network Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

The Keras wrappers require a function as an argument. This function that we must define is responsible for creating the neural network model to be evaluated.
    
Below we define the function to create the baseline model to be evaluated. It is a simple model that has a single fully connected hidden layer with the same number of neurons as input attributes (9). The network uses rectifier activation function (relu) for the hidden layer. No activation function is used for the output layer because it is a regression problem and we want to predict numerical values directly without transform.
    
The efficient ADAM optimization algorithm is used and a mean squared error loss function is optimized. 

In [ ]:
seed = 42
np.random.seed(seed)
dims = X.shape[1]
# define base model
  
def baseline_model():

    # create model
    model = Sequential()
    model.add(Dense(dims, input_shape=(dims,), init = 'normal', activation='relu'))
    model.add(Dense(9, init = 'normal', activation='relu'))
    model.add(Dense(1, init = 'normal'))
    # compile model
#     sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mse', optimizer = 'adam', metrics=['mape'])
#     model.summary()
    return model


In [ ]:
# Function to plot history of model learning 
def plot_history(network_history):
    
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('MAPE')
    plt.plot(network_history.history['mean_absolute_percentage_error'])
    plt.plot(network_history.history['val_mean_absolute_percentage_error'])
    plt.legend(['Training', 'Validation'])

We also initialize the random number generator with a constant random seed, a process we will repeat for each model evaluated in this tutorial. This is an attempt to ensure we compare models consistently.

In [ ]:
# fix random seed for reproducibility
np.random.seed(seed)
# evaluate model 
early_stop = EarlyStopping(monitor='val_mean_absolute_percentage_error', patience=50, verbose=0)
estimator = KerasRegressor(build_fn=baseline_model, epochs=1000, batch_size=128, verbose=0,
                          callbacks=[early_stop])

**Train and Validation Split**

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split( X, Y, test_size=0.20, random_state=42)

In [ ]:
history = estimator.fit(Xtrain, Ytrain, validation_data=(Xtest, Ytest))

In [ ]:
plot_history(history)

** Cross Validation**

The final step is to evaluate this baseline model. We will use 3-fold cross validation to evaluate the model.

As it is not implemented before hand, we will write our Mean Absolute Percentage Error function and 20% error quartile function to use in `cross_val_score function` of scikit-learn library. 

In [ ]:
from sklearn.metrics import make_scorer

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    try:
        return np.mean(np.abs((y_true - y_pred) / np.abs(y_true)) * 100)
    except:
        return np.mean((np.abs(y_true - y_pred)+0.001 ) / (np.abs(y_true)+0.001) * 100)

def absolute_twenty_percent_error_quartile(y_true, y_pred): 
    
    pred = y_pred.reshape(-1,1) 
    actual = y_true.reshape(-1,1)
    error = np.abs(pred-actual)/np.abs(actual)
    return (error[error<=0.2].shape[0]/actual.shape[0])*100
    
mape = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
twentyPercentErrorQuantile = make_scorer(absolute_twenty_percent_error_quartile, greater_is_better=True)

In [ ]:
kfold = StratifiedKFold(n_splits=3, random_state=seed)
estimator2 = KerasRegressor(build_fn=baseline_model, epochs=800, batch_size=128, verbose=0)
results = np.abs(cross_val_score(estimator2, X, Y, cv=kfold, scoring = mape))
print(results)
print()
print("Cross Validation Mean Absolute Percentage error: %.2f " % (results.mean()))

In [ ]:
#  kfold = StratifiedKFold(n_splits=3, random_state=seed)
# results = np.abs(cross_val_score(estimator, X, Y, cv=kfold, scoring = twentyPercentErrorQuantile))
# print(results)
# print()
# print("Cross Validation 20 percent error quartile: %.2f " % (results.mean()))

**All sample MAPE error and 20% error quartile score**

In [ ]:
mean_absolute_percentage_error(estimator.predict(X),Y)

In [ ]:
absolute_twenty_percent_error_quartile(estimator.predict(X),Y)

**Out of Sample sample MAPE error and 20% error quartile score**

In [ ]:
mean_absolute_percentage_error(estimator.predict(Xtest),Ytest)

In [ ]:
absolute_twenty_percent_error_quartile(estimator.predict(Xtest),Ytest)

## 3. Modeling The Standardized Dataset

An important concern with our house price dataset is that the input attributes all vary in their scales because they measure different quantities. 

It is almost always good practice to prepare our data before modeling it using a neural network model for optimization algorithms' efficiency.  

Continuing on from the above baseline model, we can re-evaluate the same model using a scaled version (0-1) of the input dataset. 

** It is important that we should not use the test data for scaling process other wise it leads to information leakage from testset**


In [ ]:

# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=400, batch_size=32, verbose=0)))
pipeline = Pipeline(estimators)

# using 3-fold cross validation again
kfold = StratifiedKFold(n_splits=3, random_state=seed)
results = np.abs(cross_val_score(pipeline, X, Y, cv=kfold, scoring = mape))
print(results)
print()
print("Cross Validation Mean Absolute Percentage error: %.2f " % (results.mean()))

**All sample MAPE error and 20% error quartile score**

In [ ]:
pipeline.fit(X,Y)

In [ ]:
mean_absolute_percentage_error(pipeline.predict(X),Y)

In [ ]:
absolute_twenty_percent_error_quartile(pipeline.predict(X),Y)

**Out of Sample sample MAPE error and 20% error quartile score**

In [ ]:
pipeline = Pipeline(estimators)
pipeline.fit(Xtrain,Ytrain)

In [ ]:
mean_absolute_percentage_error(pipeline.predict(Xtest),Ytest)

In [ ]:
absolute_twenty_percent_error_quartile(pipeline.predict(Xtest),Ytest)

A further extension would be to similarly apply a rescaling to the output variable such as normalizing it to the range of 0-1 and use a Sigmoid or similar activation function on the output layer to narrow output predictions to the same range.

## 4. Tune The Neural Network Topology

There are many concerns that can be optimized for a neural network model.
Perhaps the most important one is the structure of the network itself, including the number of layers and the number of neurons in each layer.  

In this section, we will evaluate two additional network topologies in an effort to further improve the performance of the model. We will look at both a deeper and a wider network topology.

### 4.1. Evaluate a Deeper Network Topology

One way to improve the performance a neural network is to add more layers. This might allow the model to extract and recombine higher order features embedded in the data.  

In this section we will evaluate the effect of adding one more hidden layer to the model. This new layer will have about half the number of neurons. Because there are heuristics that suggest that number of neurons in the next layer should not exceed the half of the previuos layer's number of neurons.

In [ ]:
# define the model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(dims, input_shape=(dims,), init = 'normal', activation='relu'))
    model.add(Dense(9, init = 'normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, init = 'normal'))
    # compile model
#     sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mse', optimizer = 'adam', metrics=['mape'])
#     model.summary()
    return model

We will follow the same methodology as above;

In [ ]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=400, batch_size=32, verbose=0)))
pipeline = Pipeline(estimators)

# using 3-fold cross validation again
kfold = StratifiedKFold(n_splits=3, random_state=seed)
results = np.abs(cross_val_score(pipeline, X, Y, cv=kfold, scoring = mape))
print(results)
print()
print("Cross Validation Mean Absolute Percentage error: %.2f " % (results.mean()))

### 4.2. Evaluate a Wider Network Topology

Another approach to increasing the representational capability of the model is to create a wider network.  
In this section we evaluate the effect of keeping a shallow network architecture and nearly doubling the number of neurons in the one hidden layer.  
Here, we have increased the number of neurons in the hidden layer compared to the baseline model from 9 to 15.

In [ ]:
# define wider model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(dims, input_shape=(dims,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # compile model
#     sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mse', optimizer = 'adam', metrics=['mape'])
#     model.summary()
    return model

Again evaluating the model structure in same way;

In [ ]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=400, batch_size=32, verbose=0)))
pipeline = Pipeline(estimators)

# using 3-fold cross validation again
kfold = StratifiedKFold(n_splits=3, random_state=seed)
results = np.abs(cross_val_score(pipeline, X, Y, cv=kfold, scoring = mape))
print(results)
print()
print("Cross Validation Mean Absolute Percentage error: %.2f " % (results.mean()))

** We saw that deeper model is the best of the three.**

We may want to add another layer

In [ ]:
# define the model
def larger_model2():
    # create model
    model = Sequential()
    model.add(Dense(dims, input_shape=(dims,), init = 'normal', activation='relu'))
    model.add(Dense(9, init = 'normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, init = 'normal'))
    # compile model
#     sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mse', optimizer = 'adam', metrics=['mape'])
#     model.summary()
    return model

We will follow the same methodology as above;

In [ ]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model2, epochs=400, batch_size=32, verbose=0)))
pipeline = Pipeline(estimators)

# using 3-fold cross validation again
kfold = StratifiedKFold(n_splits=3, random_state=seed)
results = np.abs(cross_val_score(pipeline, X, Y, cv=kfold, scoring = mape))
print(results)
print()
print("Cross Validation Mean Absolute Percentage error: %.2f " % (results.mean()))

That did not improve our model